In [1]:
# input to the this file is the final merged (stage 3.csv) file which is used to get the charts 
import os.path
import sys
import numpy as np
import pandas as pd
import math
import decimal

In [2]:
# read the final merged dataset file
path = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\Dependent Datasets'
os.chdir(path)

df = pd.read_csv('merged_df_3.csv')
modes = df['Mode'].unique()
modes.sort()
colnames = ['UPT_ADJ', 'VRM_ADJ', 'VRH_ADJ', 'FARE_TOTAL', 'FARE_per_UPT_2018', 'AVG_SPEED']
for _mode in modes:    
    for col in colnames:
        colName = _mode + "_" + col
        df[colName] = np.where(df['Mode'] ==str(_mode), df[str(col)],0)
        df[colName+"_diff"]=0.00
        df[colName+"_log"]=0.00
        df["PCT_"+colName+"_rel_diff"]=0.00
        df[colName+"_scale"]=0.00
        

df.drop(columns=['Unnamed: 0', 'NEW_CBSA'], inplace=True)        
df.to_csv('modified_dataset.csv')
print("Success")

Success


In [3]:
# get unique values of years and city_ids
_df = pd.read_csv('modified_dataset.csv')
_df2 = _df.drop_duplicates(['CBSA','Mode','Year'])
cbsas = _df2['CBSA'].unique()
modes = _df2['Mode'].unique()
years = _df2['Year'].unique()
df2 = _df2.set_index(['CBSA','Mode','Year'])
# df2.head()

for cbsa in cbsas:
    for mode in modes: 
        for year in years: 
            if df2.index.contains((cbsa,mode,year)): 
                if df2.index.contains((cbsa,mode,year-1)):
                    colnames = ['UPT_ADJ', 'VRM_ADJ', 'VRH_ADJ', 'FARE_TOTAL', 'FARE_per_UPT_2018', 'AVG_SPEED']
                    for col in colnames:
                        cName = str(mode)+"_"+str(col)
                        colName = cName+'_diff'
                        logName = cName+'_log'
                        reldiff = "PCT_"+cName+"_rel_diff"
                        if (df2.loc[(cbsa,mode,year),cName]>0):
                            # get the difference colmn populated
                            df2.loc[(cbsa,mode,year),colName] = df2.loc[(cbsa,mode,year),cName] - df2.loc[(cbsa,mode,year-1),cName]
                            # get the percent relative difference colmn populated
                            df2.loc[(cbsa,mode,year),reldiff] = (df2.loc[(cbsa,mode,year),colName]*(100))/(df2.loc[(cbsa,mode,year-1),cName])
                            # get the scale colmn populated
                            df2[cName+"_scale"] = df2.loc[(cbsa,mode,year),reldiff]*(df2.loc[(cbsa,mode,year-1),cName])                        
                            # get the lograthmic difference colmn populated
                            if (df2.loc[(cbsa,mode,year),cName]>0 and df2.loc[(cbsa,mode,year-1),cName]>0):
                                df2.loc[(cbsa,mode,year),logName] = np.log(df2.loc[(cbsa,mode,year),cName]) - np.log(df2.loc[(cbsa,mode,year-1),cName])
                        
                        
df2.to_csv('modified_dataset_2.csv')                       
print("Sucess")                    

C:\Users\vsgo222\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\vsgo222\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  del sys.path[0]
C:\Users\vsgo222\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  
C:\Users\vsgo222\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2855: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures

Sucess
